In [6]:
#!git clone https://github.com/githubharald/CTCDecoder.git

In [7]:
import albumentations
import torch
import numpy as np
from torchvision.utils import make_grid
from PIL import Image
from PIL import ImageFile
import matplotlib.pyplot as plt
from tqdm import tqdm
import string
from skimage import io
import os
import glob

from ctc_decoder import best_path, beam_search
from torchvision import transforms
import torch
import numpy as np
import pandas as pd
from torch.nn.utils.rnn import pack_sequence
from torch.nn.utils.rnn import pad_sequence
import torchvision.models as models
from torch.utils.model_zoo import load_url
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import metrics
import cv2
import torch
from  torch import nn
from torch.nn import functional as F
from pprint import  pprint

In [8]:
aymen_csv = pd.read_csv('resWords.csv')
aymen_csv["image_path"] = aymen_csv["IMAGE_ID"].apply(lambda x: f"Words/"+x)
upper = list(string.ascii_uppercase)
numbers = [i for i in range(10)]
aymen_csv["LABEL TYPE"] = aymen_csv["LABEL"].apply(lambda x: "phrase" if x[0] in upper else "numbers")
train_csv = aymen_csv
train_csv = train_csv[['IMAGE_ID', 'LABEL','image_path']]
train_csv = train_csv.reset_index(drop=True)
train_csv = train_csv[~train_csv['IMAGE_ID'].isin(['Amount in numbers_000007.jpg',
                              'Amount in numbers_000017.jpg', 
                              'Amount in numbers_000038.jpg',
                              'Amount in numbers_000043.jpg',
                              'Amount in numbers_000057.jpg',
                              'Amount in numbers_000330.jpg',
                              'Amount in numbers_000477.jpg',
                              'Amount in numbers_000590.jpg', 
                              'Amount in numbers_000654.jpg', 
                              'Amount in numbers_000738.jpg',
                              'Amount in numbers_000917.jpg',
                              'Amount in numbers_001238.jpg', 
                              'Amount in numbers_001274.jpg', 
                              'Amount in numbers_001348.jpg', 'Amount in numbers_001377.jpg', 'Amount in numbers_001439.jpg', 'Amount in numbers_001442.jpg', 'Amount in numbers_001616.jpg', 'Amount in numbers_001753.jpg', 'Amount in numbers_001804.jpg', 'Amount in numbers_001969.jpg', 'Amount in numbers_002103.jpg', 'Amount in numbers_002248.jpg', 'Amount in numbers_002253.jpg', 'Amount in numbers_002502.jpg', 'Amount in numbers_002523.jpg', 'Amount in numbers_002581.jpg', 'Amount in numbers_002639.jpg', 'Amount in numbers_002915.jpg', 'Amount in numbers_002951.jpg', 'Amount in numbers_003026.jpg', 'Amount in numbers_003059.jpg', 'Amount in numbers_003172.jpg', 'Amount in numbers_003258.jpg', 'Amount in numbers_003527.jpg', 'Amount in numbers_003570.jpg', 'Amount in numbers_003638.jpg', 'Amount in numbers_003700.jpg', 'Amount in numbers_003743.jpg', 'Amount in numbers_003927.jpg', 'Amount in numbers_003949.jpg', 'Amount in numbers_003957.jpg', 'Amount in numbers_004016.jpg', 'Amount in numbers_004045.jpg', 'Amount in numbers_004125.jpg', 'Amount in numbers_004144.jpg', 'Amount in numbers_004184.jpg', 'Amount in numbers_004196.jpg', 'Amount in numbers_004261.jpg', 'Amount in numbers_004315.jpg', 'Amount in numbers_004328.jpg', 'Amount in numbers_004366.jpg', 'Amount in numbers_004417.jpg', 'Amount in numbers_004477.jpg', 'Amount in numbers_004502.jpg', 'Amount in numbers_004622.jpg', 'Amount in numbers_004645.jpg', 'Amount in numbers_004679.jpg', 'Amount in numbers_004906.jpg', 'Amount in numbers_004993.jpg', 'Amount in numbers_005037.jpg', 'Amount in numbers_005190.jpg', 'Amount in numbers_005203.jpg', 'Amount in numbers_005247.jpg', 'Amount in numbers_005528.jpg', 'Amount in numbers_005573.jpg', 'Amount in numbers_005645.jpg', 'Amount in numbers_005696.jpg', 'Amount in numbers_005776.jpg', 'Amount in numbers_005799.jpg', 'Amount in numbers_005946.jpg', 'Amount in numbers_006073.jpg', 'Amount in numbers_006123.jpg', 'Amount in numbers_006158.jpg', 'Amount in numbers_006361.jpg', 'Amount in numbers_006406.jpg', 'Amount in numbers_006700.jpg', 'Amount in numbers_006951.jpg', 'Amount in numbers_006977.jpg'])]
train_csv = train_csv.reset_index(drop=True)
image_paths = train_csv['image_path']
image_files = train_csv['LABEL']

targets_orig = [x for x in image_files]

targets = [[c for c in x] for x in targets_orig]

targets_flat = [c for clist in targets for c in clist]

lbl_encoder = preprocessing.LabelEncoder()
lbl_encoder.fit(targets_flat)

targets_enc  = [lbl_encoder.transform(x) for x in targets]
targets_enc  = np.array(targets_enc ) 
train_csv["LABEL ENCODED"] = targets_enc

C:\Users\khamm\AppData\Local\Temp\ipykernel_17700\1842511682.py:37: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  targets_enc  = np.array(targets_enc )


In [9]:
class config:
    BATCH_SIZE = 64
    IMAGE_WIDTH = 500
    IMAGE_HEIGHT = 100
    NUM_WORKERS = 2
    EPOCHS = 130
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu' )

In [10]:
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

In [11]:
train_transform = transforms.Compose([
    
    transforms.ToTensor(),
    #transforms.RandomApply([AddGaussianNoise()], p=0.15),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.15),
    transforms.Resize(size=(config.IMAGE_HEIGHT,config.IMAGE_WIDTH)),
    
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
    
])

test_transform = transforms.Compose([
    
    transforms.ToTensor(),
    transforms.Resize(size=(config.IMAGE_HEIGHT,config.IMAGE_WIDTH)),
    
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
    
])

In [12]:
class dataset:
    ImageFile.LOAD_TRUNCATED_IMAGES = True

    class ClassificationDataset:
        def __init__(self, dataframe, transform=None):
            self.dataframe = dataframe
            #self.aug = albumentations.Compose([albumentations.Normalize(always_apply=True)])
            self.transform = transform

        def __len__(self):
            return len(self.dataframe)

        def __getitem__(self, item):
            image = Image.open(self.dataframe['image_path'][item]).convert("RGB")
            targets = self.dataframe['LABEL ENCODED'][item]
            
            
            
            if self.transform:
                image = self.transform(image)
            
            image = np.array(image)
            #augmented = self.aug(image=image)
            #image = augmented["image"]

            return {
                "images": torch.tensor(image, dtype=torch.float),
                "targets": torch.tensor(targets, dtype=torch.long)
            }

In [13]:
class CaptchaModel(nn.Module):
    def __init__(self, num_chars):
        super(CaptchaModel, self).__init__()
        self.conv_1 = nn.Conv2d(3, 128, kernel_size=(3, 3), padding=(1, 1))
        self.pool_1 = nn.MaxPool2d(kernel_size=(2, 2))
        self.conv_2 = nn.Conv2d(128, 64, kernel_size=(3, 3), padding=(1, 1))
        self.pool_2 = nn.MaxPool2d(kernel_size=(2, 2))

        self.linear1 = nn.Linear(1600, 64)
        self.drop_1 = nn.Dropout(0.2)

        self.gru = nn.GRU(64, 32, bidirectional=True, num_layers=2, dropout=0.25, batch_first=True)
        self.output = nn.Linear(64, num_chars + 1)

    def forward(self, images,lens, targets=None):
        bs, c, h, w = images.size()
        #print(bs, c, h, w)
        x = F.relu(self.conv_1(images))
        x = self.pool_1(x)
        x = F.relu(self.conv_2(x))
        x = self.pool_2(x) # 1, 64, 18, 75
        x = x.permute(0, 3, 1, 2) # 1, 75 , 64, 18
        x = x.view(bs, x.size(1), -1)
        x = self.linear1(x)
        x = self.drop_1(x)
        #print(x.size()) # torch.Size([1, 75, 64]) -> we have 75 time steps and for each time step we have 64 values
        x, _ = self.gru(x)
        x = self.output(x)
        x = x.permute(1, 0, 2) # bs, time steps, values -> CTC LOSS expects it to be

        if targets is not None:
            log_probs = F.log_softmax(x, 2)
            input_lengths = torch.full(
                size=(bs,), fill_value=log_probs.size(0), dtype=torch.int32
            )

            target_lengths = torch.tensor(
                lens, dtype=torch.int32
            )
            loss = nn.CTCLoss(blank=len(lbl_encoder.classes_))(
                log_probs, targets, input_lengths, lens
            )
            return x, loss

        return x, None

In [14]:
def collate(batch):
    images, words = [b.get('images') for b in batch], [b.get('targets') for b in batch]
    images = torch.stack(images, 0)

    lens = [len(item['targets']) for item in batch]
    
    list_of_chars_digitized = [word for word in words]
    lengths = len(list_of_chars_digitized)

    lengths = torch.tensor(lengths, dtype=torch.long)
    lens = torch.tensor(lens, dtype=torch.long)

    for i in range(lengths.item()):
        targets = torch.cat(words)

    
    
    return  images,targets, lens

In [15]:
class engine:
    def train_fn(model, data_loader, optimizer):
        model.train()
        fin_loss = 0
        tk0 = tqdm(data_loader, total=len(data_loader))
        for xb, yb, lens in tk0:
 
            xb = xb.to(config.DEVICE)
            yb = yb.to(config.DEVICE)
            lens = lens.to(config.DEVICE)
            
            optimizer.zero_grad()
            _, loss = model(xb, lens, yb)
            loss.backward()
            optimizer.step()
            fin_loss += loss.item()
        return fin_loss / len(data_loader)


    def eval_fn(model, data_loader):
        model.eval()
        fin_loss = 0
        fin_preds = []
        with torch.no_grad():
            tk0 = tqdm(data_loader, total=len(data_loader))
            for xb, yb, lens in tk0:

                
                xb = xb.to(config.DEVICE)
                yb = yb.to(config.DEVICE)
                lens = lens.to(config.DEVICE)
                
                batch_preds, loss = model(xb, lens, yb)
                fin_loss += loss.item()
                fin_preds.append(batch_preds.detach().cpu())
            return fin_preds, fin_loss / len(data_loader)

In [16]:
def decode_predictions(preds, encoder):

    preds = preds.permute(1, 0, 2) 
    preds = torch.softmax(preds, 2)

    preds = torch.argmax(preds, 2)

    preds = preds.numpy()

    cap_preds = []
    
    for j in range(preds.shape[0]):
        temp = []
        for k in preds[j, :]:
            if k == len(lbl_encoder.classes_):
                temp.append("°")
            else:
                p = encoder.inverse_transform([k])[0]
                temp.append(p)
        tp = "".join(temp)
        cap_preds.append(tp)
    return cap_preds

In [17]:
def decode_predictions_2(preds, encoder):
    preds = torch.softmax(preds, 2)
    preds = preds.detach().cpu().numpy()
    for i in range(preds.shape[1]):
        aux =  preds[:,i,:] 
        print(f'Beam search: "{beam_search(aux,lbl_encoder.classes_)}"')
        break
    
    return None

In [18]:
def run_training():
    (
        train,
        test,
        _,
        test_targets_orig,
    ) = model_selection.train_test_split(
        train_csv, targets_orig, test_size=0.1, random_state=42
    )

    train = train.reset_index(drop=True)
    test = test.reset_index(drop=True)
    
    train_dataset = dataset.ClassificationDataset(
        dataframe=train,
        transform=train_transform
        )
    train_loader = torch.utils.data.DataLoader(
            train_dataset,
            batch_size=config.BATCH_SIZE,
            shuffle=True,
            collate_fn=collate
        )
    test_dataset = dataset.ClassificationDataset(
        dataframe=test,
        transform=test_transform
        
    )
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=config.BATCH_SIZE,
        shuffle=False,
        collate_fn=collate
    )

    model = CaptchaModel(num_chars=len(lbl_encoder.classes_))
    model.to(config.DEVICE)

    optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, factor=0.8, patience=5, verbose=True
    )
    best_score = 0.0 
    best_loss = np.inf

    for epoch in range(config.EPOCHS):
        train_loss = engine.train_fn(model, train_loader, optimizer)
        valid_preds, valid_loss = engine.eval_fn(model, test_loader)
        valid_cap_preds = []
        for vp in valid_preds:
            decode_predictions_2(vp ,lbl_encoder )
            break
            
        for vp in valid_preds:
            current_preds = decode_predictions(vp, lbl_encoder)
            valid_cap_preds.extend(current_preds)

        pprint(list(zip(test_targets_orig, valid_cap_preds))[0:6])
        print(f"EPOCH: {epoch}   ,    train_loss={train_loss},    valid_loss={valid_loss}")

        if valid_loss < best_loss:
            best_loss = valid_loss
            print("=> saving checkpoint")
            torch.save(model.state_dict(),'Words_best_score.pt')

In [19]:
run_training()